# CAD-DR: CAD Dimensionality Reduction

A deep convolutional autoencoder for dimensionality reduction of 3D CAD models

### Information
1. Ten-layer architecture (5 in encoder + 5 in decoder)
2. Dataset - 1000 STL files
3. Train - 800 models
4. Test - 200 models
5. Epochs - 50
6. Point cloud density - 20k points

### Metrics
1. loss: 0.0180
2. accuracy: 0.9920 (99.20%)
3. val_loss: 0.0120
4. val_accuracy: 0.9948 (99.48%)

### Encoder architecture
- Conv3D(32, (3, 3, 3), activation='selu', padding='same')
- AveragePooling3D((2, 2, 2), padding='same')
- Conv3D(16, (3, 3, 3), activation='selu', padding='same')
- AveragePooling3D((2, 2, 2), padding='same')

#### Run this cell in case you haven't installed the required libraries

In [ ]:
!pip install numpy pandas pyntcloud open3d
!pip install tensorflow[and-cuda]

### Importing all necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Conv3D, UpSampling3D, AveragePooling3D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import numpy as np
import pandas as pd
from pyntcloud import PyntCloud
import open3d as o3d
from ConversionUtils import ConversionUtils
from Visualization import Visualization

### Converting STL to point cloud

**Execute the following cell if you wish to delete all existing point cloud files in abc-dataset-ply/ directory**

In [ ]:
folder_path = "abc-dataset-ply/"
files = os.listdir(folder_path)

for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file}: {str(e)}")

**Only execute the next cell if the point cloud files do not exist in abc-dataset-ply/ directory**

In [ ]:
files = sorted(ConversionUtils.list_files_in_directory("abc-dataset-stl/"))[302]
for i in files:
    path = "abc-dataset-stl/" + i
    ConversionUtils.stl_to_ply(path, 20000)

### Converting point cloud to binary voxel arrays

In [ ]:
directory = "abc-dataset-ply/"

files = sorted([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])
dataset = []

for i in files:
    path = os.path.join(directory, i)
    binvox = ConversionUtils.convert_to_binvox(path, 64)
    dataset.append(binvox)

In [ ]:
print(len(dataset))

### Splitting dataset for training and testing

In [ ]:
dataset = np.array(dataset)

train_dataset_length = int(0.8 * len(dataset))
test_set_length = int(0.2 * len(dataset))

train_dataset = dataset[:train_dataset_length]  
test_dataset = dataset[train_dataset_length:]   
print(len(train_dataset), len(test_dataset))

input_shape = (64, 64, 64, 1)

### Encoder

In [ ]:
input_data = Input(shape=input_shape)
x = Conv3D(32, (3, 3, 3), activation='selu', padding='same')(input_data)
x = AveragePooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(16, (3, 3, 3), activation='selu', padding='same')(x)
encoded = AveragePooling3D((2, 2, 2), padding='same')(x)

### Decoder

In [ ]:
x = Conv3D(16, (3, 3, 3), activation='selu', padding='same')(encoded)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3D(32, (3, 3, 3), activation='selu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
decoded = Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(x)

### Callbacks

#### Early Stopping

In [ ]:
early_stopping = EarlyStopping(
    monitor="loss",
    min_delta=0.0001,
    patience=5,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

#### Checkpoint

In [ ]:
checkpoint_filepath = 'checkpoints/checkpoint.weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

### Training autoencoder, prediction done on test dataset

In [ ]:
BATCH_SIZE = 10
EPOCHS = 50

In [ ]:
autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

autoencoder.fit(train_dataset, train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_dataset, test_dataset), callbacks=[early_stopping, model_checkpoint_callback])

In [ ]:
reconstructed_data = autoencoder.predict(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
encoder = Model(autoencoder.input, autoencoder.layers[5].output)
encoded_data = encoder.predict(test_dataset, batch_size=BATCH_SIZE)

### Saving autoencoder and encoder

In [ ]:
autoencoder.save("saved-models/autoencoder.keras")
encoder.save("saved-models/encoder.keras")

### Dimensions

In [ ]:
print("Shape of input data: ", test_dataset[0].shape)
print("Shape of encoded data: ", encoded_data[0].shape)
print("Shape of reconstructed data: ", reconstructed_data[0].shape)

### Sample reconstruction from test dataset

In [ ]:
import matplotlib.pyplot as plt

index = 55 # change this value to visualize different models and their reconstructions

original_sample = test_dataset[index]

reconstructed_sample = reconstructed_data[index].reshape(64, 64, 64)
threshold = 0.35
reconstructed_sample = (reconstructed_sample > threshold).astype(int)

In [ ]:
Visualization.matplotlib_visualize_original(original_sample)

In [ ]:
Visualization.matplotlib_visualize_reconstructed(reconstructed_sample)

In [ ]:
Visualization.open3d_visualize_original(original_sample)

In [ ]:
Visualization.open3d_visualize_reconstructed(reconstructed_sample)

### Visualizing encoded data

In [ ]:
sample_encoded_data = encoded_data[index]
print(sample_encoded_data.shape)

In [ ]:
print(sample_encoded_data)

In [ ]:
threshold = 0.35
binary_arrays = (sample_encoded_data >= threshold).astype(int)
print(binary_arrays)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

colors = plt.cm.get_cmap('tab20', len(binary_arrays))

for i, binary_array in enumerate(binary_arrays):
    x, y, z = np.where(binary_array == 1)
    ax.scatter(x, y, z, c=colors(i), marker='o', s=20, label=f'Channel {i + 1}')

ax.set_box_aspect([1, 1, 1])

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.legend()

plt.show()